# Model creation stage III (4x8)

In [1]:
%load_ext autoreload
%autoreload 2

# Setup

In [37]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

import os
import time
import json
import csv
import os
import time
import json
import socket
import serial
from tools.mlx import *
import multiprocessing
import numpy as np
import cv2
import time
import os
import dill

GAIN = 7 # Gain setting (same as firmware)
RESOLUTION = 0 # Resolution setting (same as firmware)
BAUD = 115200 # Baud rate
COM = '/dev/ttyACM0' # Serial port
ENABLE_WS = True # Enable WebSocket server. Disable if not using websocket. Script will crash otherwise.
NR_OF_SENSORS = 32 # Number of sensors
TEMP_COMP = True
ANGLE = 91.82
save_path = os.path.join(os.getcwd(), '..', 'models', "4x8")

models = []
    
for i in range(NR_OF_SENSORS):

    with open(os.path.join(save_path, f"taxel{i}"), 'rb') as f:
        model = dill.load(f)
        models.append(model)

In [38]:
ser = serial.Serial(COM, BAUD, timeout=1)
ser.close()

In [39]:
udp_server_address = ('localhost', 9870)
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [40]:
with open(os.path.join(save_path, f"stageII"), 'rb') as f:
    stage_II_model = dill.load(f)

In [51]:
def readout_for_t_seconds(t):
    
    Gs = []
    
    t0 = time.time()
    t1 = time.time()
    is_first = True

    # Stuff for unwrapping
    prev_x = [0] * NR_OF_SENSORS
    prev_y = [0] * NR_OF_SENSORS
    prev_z = [0] * NR_OF_SENSORS
    
    cycle_x = [0] * NR_OF_SENSORS
    cycle_y = [0] * NR_OF_SENSORS
    cycle_z = [0] * NR_OF_SENSORS    
    
    # Debiasing stuff
    bias_x = [0] * NR_OF_SENSORS
    bias_y = [0] * NR_OF_SENSORS
    bias_z = [0] * NR_OF_SENSORS
    bias_counter = 0
    
    # Feature vectors
    X = np.zeros((NR_OF_SENSORS, 3))
    
    ser.open()
    while time.time() - t0 < t:    
        
        data = ser.read(1)
        if data == b'\xAA':
            
            data_bytes = ser.read(6 * NR_OF_SENSORS)
            
            if is_first:
                t1 = time.time()
            
            row = {}

            for i in range(NR_OF_SENSORS):
                
                # Encoding
                x = ~ ( (data_bytes[i * 3 * 2] << 8) + data_bytes[i * 3 * 2 + 1] )
                y = ~ ( (data_bytes[i * 3 * 2 + 2] << 8) + data_bytes[i * 3 * 2 + 3] )
                z = ~ ( (data_bytes[i * 3 * 2 + 4] << 8) + data_bytes[i * 3 * 2 + 5] )

                # Unwrapping
                if not is_first:
                    
                    # Check for large jumps and correct them
                    if abs(x - prev_x[i]) > 2 ** 15:
                        cycle_x[i] += 1 if x < prev_x[i] else -1
                        
                    if abs(y - prev_y[i]) > 2 ** 15:
                        cycle_y[i] += 1 if y < prev_y[i] else -1
                    
                    if abs(z - prev_z[i]) > 2 ** 15:
                        cycle_z[i] += 1 if z < prev_z[i] else -1      
                
                prev_x[i] = x
                prev_y[i] = y
                prev_z[i] = z
                
                x += cycle_x[i] * 2 ** 16
                y += cycle_y[i] * 2 ** 16
                z += cycle_z[i] * 2 ** 16                    
                                        
                is_first = False

                # Conversion based on sensor settings (to uT)
                x *= mlx90393_lsb_lookup[0][GAIN][RESOLUTION][0] 
                y *= mlx90393_lsb_lookup[0][GAIN][RESOLUTION][0] 
                z *= mlx90393_lsb_lookup[0][GAIN][RESOLUTION][1] 
                
                # x += mlx90393_lsb_lookup[0][GAIN][RESOLUTION][0] * 2 ** 16
                # y += mlx90393_lsb_lookup[0][GAIN][RESOLUTION][0] * 2 ** 16
                # z += mlx90393_lsb_lookup[0][GAIN][RESOLUTION][1] * 2 ** 16
                
                # Global transformation
                X_i = x * np.cos(np.deg2rad(ANGLE)) - y * np.sin(np.deg2rad(ANGLE))
                Y_i = x * np.sin(np.deg2rad(ANGLE)) + y * np.cos(np.deg2rad(ANGLE))
                
                x = X_i.copy()
                y = Y_i.copy()
                
                #Local transformation
                z = 1 / z

                # Debiasing
                if bias_counter < 100:
                    bias_x[i] += x
                    bias_y[i] += y
                    bias_z[i] += z
                    continue
                
                elif bias_counter == 100:
                    bias_x[i] /= 100
                    bias_y[i] /= 100
                    bias_z[i] /= 100
                    print("Done")
                
                x -= bias_x[i]
                y -= bias_y[i]
                z -= bias_z[i]
                
                row[f"X{i}"] = x
                row[f"Y{i}"] = y
                row[f"Z{i}"] = z        
                
                # Put in feature vector
                X[i] = [x, y, z]
            
            bias_counter += 1
            
            # Predict
            G, F_comp = stage_II_model.predict_single(X)
            G = G[0]
            
            for j in range(0, NR_OF_SENSORS, 3):
                row[f"G{j}_x"] = G[j]
                row[f"G{j}_y"] = G[j + 1]
                row[f"G{j}_z"] = G[j + 2]
                
            
             
            print(row[f"G{0}_z"])                
            
            # json_data = json.dumps(row)
            # udp_socket.sendto(json_data.encode(), udp_server_address)
    
    print("FPS: ", bias_counter / (time.time() - t1))
    ser.close()

In [53]:
ser.close()
readout_for_t_seconds(30)

-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.011304518265669429
-0.0113045

In [43]:
ser.close()